**No need to run everytime, because once data is extracted**

In [ ]:
# import zipfile
# import os

# # Specify the path to the zip file
# zip_file_path = "/content/drive/MyDrive/FashionVision_Dataset.zip"  # Change this to the path of your zip file

# # Specify the directory to unzip the contents
# extract_dir = "/content/drive/MyDrive"  # Change this to the directory where you want to extract the files

# # Create the directory if it doesn't exist
# os.makedirs(extract_dir, exist_ok=True)

# # Unzip the file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# # Check the contents of the extracted directory
# extracted_files = os.listdir(extract_dir)
# print("Files extracted successfully:", extracted_files)


**Transformation and Augumentation done**

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torchvision import transforms
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir, 'images')
        self.label_dir = os.path.join(root_dir, 'labels')
        self.image_filenames = os.listdir(self.image_dir)
        self.transform = transform
        self.labels = self._read_labels()

    def _read_labels(self):
        labels = {}
        for filename in os.listdir(self.label_dir):
            if filename.endswith(".txt"):
                with open(os.path.join(self.label_dir, filename), 'r') as file:
                    label = int(file.readline().strip().split()[0])  # assuming label is the first integer in the text file
                    labels[filename[:-4]] = label  # removing '.txt' extension from filename
        return labels

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        image = cv2.imread(os.path.join(self.image_dir, img_name))
        label = self.labels[img_name[:-4]]
        if self.transform:
            image = self.transform(image)
        return image, label

# Example usage:
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy array to PIL image
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert PIL image to tensor
    # Add any additional transformations here if needed
])

root_dir = "/content/drive/MyDrive/FashionVision_Dataset"
dataset = CustomDataset(root_dir, transform=transform)


dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Now you can iterate over the dataloader to get batches of resized images and labels
for batch_idx, (images, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}:")
    print("Images shape:", images.shape)  # Shape of the batch of images (should be [batch_size, channels, height, width])
    print("Labels:", labels)  # Labels corresponding to the batch of images


In [ ]:

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [ ]:
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))

Number of training samples: 730
Number of validation samples: 183


In [ ]:
# Calculate the number of unique classes in the dataset
num_classes = len(torch.unique(torch.tensor(list(dataset.labels.values()))))
print("Number of classes in the dataset:", num_classes)


Number of classes in the dataset: 5


In [4]:

# # Define a function to display sample images
# def show_images(images, labels, class_names, num_images=5):
#     fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
#     for i in range(num_images):
#         ax = axes[i]
#         ax.imshow(np.transpose(images[i], (1, 2, 0)))
#         ax.set_title(class_names[labels[i]])
#         ax.axis("off")
#     plt.show()

# # Get a batch of sample images and labels
# sample_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
# sample_batch = next(iter(sample_loader))
# sample_images, sample_labels = sample_batch

# # Display the sample images
# show_images(sample_images, sample_labels, dataset.class_names)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def show_images(images, labels, num_images=4):
    """
    Display images from a batch along with their corresponding labels.

    Args:
        images (torch.Tensor): Batch of images with shape (batch_size, channels, height, width).
        labels (torch.Tensor): Batch of labels corresponding to the images.
        num_images (int): Number of images to display. Default is 4.
    """
    # Ensure num_images is not greater than the batch size
    num_images = min(num_images, len(images))

    # Create a grid of subplots
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    image_count=0
    for i in range(num_images):
        image_count+=1
        # Convert the tensor image to numpy array and transpose to (height, width, channels)
        image_np = np.transpose(images[i].numpy(), (1, 2, 0))

        # Display the image
        axes[i].imshow(image_np)
        axes[i].set_title(f"Label: {labels[i]}")
        axes[i].axis('off')
    print("Number of images:", image_count)
    #plt.show()

# Example usage:
batch_count=0
for batch_idx, (images, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}:")
    #batch_count+=1
    show_images(images, labels)
print("Batches: ", batch_count)

**Model Implementation**

In [3]:
domain_x_path = "/content/sample_data/domain_x_dataset"  # Fabric patches
domain_y_path = "/content/sample_data/domain_y_dataset"  # Full dress images

domain_x_dataset = datasets.ImageFolder(domain_x_path, transform=transform)
domain_y_dataset = datasets.ImageFolder(domain_y_path, transform=transform)

domain_x_loader = DataLoader(domain_x_dataset, batch_size=1, shuffle=True)
domain_y_loader = DataLoader(domain_y_dataset, batch_size=1, shuffle=True)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # Add your layers here
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            # Add your layers here
        )

    def forward(self, x):
        return self.model(x)

G = Generator()
F = Generator()
D_X = Discriminator()
D_Y = Discriminator()

G_optimizer = optim.Adam(G.parameters(), lr=0.0002)
F_optimizer = optim.Adam(F.parameters(), lr=0.0002)
D_X_optimizer = optim.Adam(D_X.parameters(), lr=0.0002)
D_Y_optimizer = optim.Adam(D_Y.parameters(), lr=0.0002)

adversarial_loss = nn.MSELoss()
cycle_loss = nn.L1Loss()

num_epochs = 100

for epoch in range(num_epochs):
    for (x_batch, _), (y_batch, _) in zip(domain_x_loader, domain_y_loader):
        # Train Discriminator D_X
        D_X_optimizer.zero_grad()
        real_x = x_batch
        fake_x = F(y_batch)
        real_x_output = D_X(real_x)
        fake_x_output = D_X(fake_x.detach())
        D_X_loss = adversarial_loss(real_x_output, torch.ones_like(real_x_output)) + adversarial_loss(fake_x_output, torch.zeros_like(fake_x_output))
        D_X_loss.backward()
        D_X_optimizer.step()

        # Train Discriminator D_Y
        D_Y_optimizer.zero_grad()
        real_y = y_batch
        fake_y = G(x_batch)
        real_y_output = D_Y(real_y)
        fake_y_output = D_Y(fake_y.detach())
        D_Y_loss = adversarial_loss(real_y_output, torch.ones_like(real_y_output)) + adversarial_loss(fake_y_output, torch.zeros_like(fake_y_output))
        D_Y_loss.backward()
        D_Y_optimizer.step()

        # Train Generators G and F
        G_optimizer.zero_grad()
        F_optimizer.zero_grad()

        fake_y = G(x_batch)
        fake_y_output = D_Y(fake_y)
        G_adversarial_loss = adversarial_loss(fake_y_output, torch.ones_like(fake_y_output))
        reconstructed_x = F(fake_y)
        cycle_x_loss = cycle_loss(reconstructed_x, x_batch)
        G_loss = G_adversarial_loss + cycle_x_loss

        G_loss.backward()
        G_optimizer.step()

        fake_x = F(y_batch)
        fake_x_output = D_X(fake_x)
        F_adversarial_loss = adversarial_loss(fake_x_output, torch.ones_like(fake_x_output))
        reconstructed_y = G(fake_x)
        cycle_y_loss = cycle_loss(reconstructed_y, y_batch)
        F_loss = F_adversarial_loss + cycle_y_loss

        F_loss.backward()
        F_optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], D_X Loss: {D_X_loss.item()}, D_Y Loss: {D_Y_loss.item()}, G Loss: {G_loss.item()}, F Loss: {F_loss.item()}")

    if epoch % 10 == 0:
        torch.save(G.state_dict(), f"G_epoch{epoch}.pth")
        torch.save(F.state_dict(), f"F_epoch{epoch}.pth")
        torch.save(D_X.state_dict(), f"D_X_epoch{epoch}.pth")
        torch.save(D_Y.state_dict(), f"D_Y_epoch{epoch}.pth")


Conditional GANs

In [3]:
domain_x_path = "/content/sample_data/domain_x_dataset"
domain_y_path = "/content/sample_data/domain_y_dataset"


domain_x_dataset = datasets.ImageFolder(domain_x_path, transform=transform)
domain_y_dataset = datasets.ImageFolder(domain_y_path, transform=transform)

domain_x_loader = DataLoader(domain_x_dataset, batch_size=1, shuffle=True)
domain_y_loader = DataLoader(domain_y_dataset, batch_size=1, shuffle=True)

class Generator(nn.Module):
    def __init__(self, input_channels=3, output_channels=3, condition_dim=10):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_channels + condition_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, output_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x, condition):
        input_data = torch.cat((x, condition), dim=1)
        return self.model(input_data)

class Discriminator(nn.Module):
    def __init__(self, input_channels=3, condition_dim=10):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_channels + condition_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(128 * 64 * 64, 1),
            nn.Sigmoid()
        )

    def forward(self, x, condition):
        input_data = torch.cat((x, condition), dim=1)
        return self.model(input_data)

G = Generator()
D = Discriminator()

generator = Generator(input_channels=3, output_channels=3, condition_dim=10)
discriminator = Discriminator(input_channels=3, condition_dim=10)

fake_image = generator(input_image, condition)
discriminator_output = discriminator(fake_image, condition)

adversarial_loss = nn.BCELoss()

num_epochs = 100

for epoch in range(num_epochs):
    for (x_batch, _), (y_batch, _) in zip(domain_x_loader, domain_y_loader):
        condition = generate_condition()  # Customize condition generation

        D_optimizer.zero_grad()
        real_y = y_batch
        fake_y = G(x_batch, condition)
        real_y_output = D(real_y, condition)
        fake_y_output = D(fake_y.detach(), condition)
        D_loss = adversarial_loss(real_y_output, torch.ones_like(real_y_output)) + adversarial_loss(fake_y_output, torch.zeros_like(fake_y_output))
        D_loss.backward()
        D_optimizer.step()

        G_optimizer.zero_grad()
        fake_y = G(x_batch, condition)
        fake_y_output = D(fake_y, condition)
        G_loss = adversarial_loss(fake_y_output, torch.ones_like(fake_y_output))
        G_loss.backward()
        G_optimizer.step()

        print(f"Epoch [{epoch + 1}/{num_epochs}], D Loss: {D_loss.item()}, G Loss: {G_loss.item()}")

    if epoch % 10 == 0:
        torch.save(G.state_dict(), f"G_epoch{epoch}.pth")
        torch.save(D.state_dict(), f"D_epoch{epoch}.pth")
